# Compagnon Immobilier - Extraction des Données

## Notebook 02 : Extraction via l'API INSEE

---

Ce notebook documente le processus d'extraction des données économiques et démographiques depuis l'API INSEE via la bibliothèque **pynsee**.

## 1. Configuration et Imports

In [ ]:
# Imports standards
import os
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Imports pynsee
from pynsee.macrodata import get_dataset_list, get_series_list, get_series
from pynsee.localdata import get_local_data

# Configuration des chemins
PROJECT_ROOT = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(PROJECT_ROOT, 'Economie_Demographie', 'data')

# Créer le dossier si nécessaire
os.makedirs(DATA_DIR, exist_ok=True)

print(f"Dossier de données : {DATA_DIR}")
print(f"Date d'exécution : {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## 2. Comprendre l'API pynsee

### 2.1 Types de données disponibles

L'API pynsee propose deux types principaux de données :

| Module | Fonction | Description |
|--------|----------|-------------|
| `macrodata` | `get_dataset_list()` | Liste des datasets macro disponibles |
| `macrodata` | `get_series_list(dataset)` | Séries temporelles d'un dataset |
| `macrodata` | `get_series(series_id)` | Données d'une série spécifique |
| `localdata` | `get_local_data()` | Données locales (départements, communes) |

In [2]:
# Explorer les datasets disponibles
datasets = get_dataset_list()
print(f"Nombre total de datasets : {len(datasets)}")
print(f"\nColonnes disponibles : {datasets.columns.tolist()}")
print("\n" + "="*80)
print("Premiers datasets :")
print("="*80)
datasets[['id', 'Name.fr', 'n_series']].head(10)

Nombre total de datasets : 226

Colonnes disponibles : ['id', 'Name.fr', 'Name.en', 'url', 'n_series']

Premiers datasets :


,id,Name.fr,n_series
0,BALANCE-PAIEMENTS,Balance des paiements,197
1,CHOMAGE-TRIM-NATIONAL,"Chômage, taux de chômage par sexe et âge (sens...",169
2,CLIMAT-AFFAIRES,Indicateurs synthétiques du climat des affaires,3
3,CNA-2010-CONSO-MEN,Consommation des ménages - Résultats par produ...,2247
4,CNA-2010-CONSO-SI,Dépenses de consommation finale par secteur in...,1391
5,CNA-2010-CPEB,Comptes de production et d’exploitation par br...,2739
6,CNA-2010-CSI,Comptes des secteurs institutionnels - Résulta...,1173
7,CNA-2010-DEP-APU,Dépenses des administrations publiques - Résul...,4400
8,CNA-2010-DETTE-APU,Dette et déficit des administrations publiques...,72
9,CNA-2010-EMPLOI,"Emploi intérieur, durée effective travaillée e...",702


### 2.2 Recherche de datasets pertinents

Recherchons les datasets liés à notre projet immobilier :

In [3]:
# Fonction de recherche dans les datasets
def rechercher_datasets(mot_cle):
    """Recherche des datasets contenant un mot-clé."""
    mask = datasets['Name.fr'].str.lower().str.contains(mot_cle.lower(), na=False)
    resultats = datasets[mask][['id', 'Name.fr', 'n_series']]
    print(f"Recherche '{mot_cle}' : {len(resultats)} résultat(s)")
    return resultats

# Recherches pertinentes pour l'immobilier
print("=" * 80)
print("DATASETS PERTINENTS POUR L'IMMOBILIER")
print("=" * 80)

for terme in ['chomage', 'pib', 'prix', 'population', 'revenu', 'logement', 'inflation']:
    print(f"\n--- {terme.upper()} ---")
    display(rechercher_datasets(terme))

DATASETS PERTINENTS POUR L'IMMOBILIER

--- CHOMAGE ---
Recherche 'chomage' : 0 résultat(s)


,id,Name.fr,n_series



--- PIB ---
Recherche 'pib' : 3 résultat(s)


,id,Name.fr,n_series
14,CNA-2010-PIB,Produit intérieur brut (PIB) et ses composante...,34
29,CNA-2014-PIB,Produit intérieur brut (PIB) et ses composantes,25
40,CNA-2020-PIB,Produit intérieur brut (PIB) et ses composantes,21



--- PRIX ---
Recherche 'prix' : 29 résultat(s)


...

,id,Name.fr,n_series
15,CNA-2010-RDB,Revenu et pouvoir d’achat des ménages - Résult...,11
30,CNA-2014-RDB,Revenu et pouvoir d’achat des ménages,12
204,TCRED-SALAIRES-REVENUS-MEN,TCRED - Revenus disponibles des ménages,565
206,TCRED-SALAIRES-REVENUS-REV-SAL-SEXE-CS,TCRED - Revenu salarial annuel moyen selon la ...,819
207,TCRED-SALAIRES-REVENUS-STRUCTURE-RDB,TCRED - Structure du revenu disponible,565



--- LOGEMENT ---
Recherche 'logement' : 6 résultat(s)


,id,Name.fr,n_series
61,CONSTRUCTION-LOGEMENTS,Construction de logements,608
124,IPEA-2010,Indice des prix d’entretien-amélioration des l...,10
134,IPLA-IPLNA-2010,Indices des prix des logements neufs et indice...,112
135,IPLA-IPLNA-2015,Indices des prix des logements neufs et Indice...,142
167,PARC-LOGEMENTS,Estimations annuelles du parc de logements (EAPL),26
183,TCRED-CONDITIONSDEVIE-LOG-SOC,TCRED - Logements sociaux,468



--- INFLATION ---
Recherche 'inflation' : 0 résultat(s)


,id,Name.fr,n_series


---

## 3. Extraction des Données Macroéconomiques

### 3.1 PIB - Produit Intérieur Brut

In [4]:
# Exploration du dataset PIB
dataset_pib = 'CNT-2020-PIB-EQB-RF'

# Lister les séries disponibles
series_pib = get_series_list(dataset_pib)
print(f"Dataset : {dataset_pib}")
print(f"Nombre de séries : {len(series_pib)}")
print(f"\nColonnes : {series_pib.columns.tolist()}")
series_pib.head()

Dataset : CNT-2020-PIB-EQB-RF
Nombre de séries : 68

Colonnes : ['DATASET', 'IDBANK', 'KEY', 'FREQ', 'INDICATEUR', 'SECT_INST', 'OPERATION', 'CNA_PRODUIT', 'NATURE', 'REF_AREA', 'VALORISATION', 'UNIT_MEASURE', 'CORRECTION', 'SERIE_ARRETEE', 'FREQ_label_fr', 'FREQ_label_en', 'INDICATEUR_label_fr', 'INDICATEUR_label_en', 'SECT_INST_label_fr', 'SECT_INST_label_en', 'OPERATION_label_fr', 'OPERATION_label_en', 'CNA_PRODUIT_label_fr', 'CNA_PRODUIT_label_en', 'NATURE_label_fr', 'NATURE_label_en', 'REF_AREA_label_fr', 'REF_AREA_label_en', 'VALORISATION_label_fr', 'VALORISATION_label_en', 'UNIT_MEASURE_label_fr', 'UNIT_MEASURE_label_en', 'CORRECTION_label_fr', 'CORRECTION_label_en', 'SERIE_ARRETEE_label_fr', 'SERIE_ARRETEE_label_en']


,DATASET,IDBANK,KEY,FREQ,INDICATEUR,SECT_INST,OPERATION,CNA_PRODUIT,NATURE,REF_AREA,...,REF_AREA_label_fr,REF_AREA_label_en,VALORISATION_label_fr,VALORISATION_label_en,UNIT_MEASURE_label_fr,UNIT_MEASURE_label_en,CORRECTION_label_fr,CORRECTION_label_en,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en
0,CNT-2020-PIB-EQB-RF,011794891,T.CNT-EQUILIBRE_PIB.S15.P51P.D-CNT.VALEUR_ABSO...,T,CNT-EQUILIBRE_PIB,S15,P51P,D-CNT,VALEUR_ABSOLUE,FE,...,France,France,Valeurs aux prix courants,Values at current prices,euros,euros,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no
1,CNT-2020-PIB-EQB-RF,011794888,T.CNT-EQUILIBRE_PIB.S0.P51.D-CNT.VALEUR_ABSOLU...,T,CNT-EQUILIBRE_PIB,S0,P51,D-CNT,VALEUR_ABSOLUE,FE,...,France,France,Valeurs aux prix courants,Values at current prices,euros,euros,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no
2,CNT-2020-PIB-EQB-RF,011794889,T.CNT-EQUILIBRE_PIB.S13.P51G.D-CNT.VALEUR_ABSO...,T,CNT-EQUILIBRE_PIB,S13,P51G,D-CNT,VALEUR_ABSOLUE,FE,...,France,France,Valeurs aux prix courants,Values at current prices,euros,euros,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no
3,CNT-2020-PIB-EQB-RF,011794890,T.CNT-EQUILIBRE_PIB.S14.P51M.D-CNT.VALEUR_ABSO...,T,CNT-EQUILIBRE_PIB,S14,P51M,D-CNT,VALEUR_ABSOLUE,FE,...,France,France,Valeurs aux prix courants,Values at current prices,euros,euros,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no
4,CNT-2020-PIB-EQB-RF,011794892,T.CNT-EQUILIBRE_PIB.S15.P3.SO.RATIO.FE.L.SO.CV...,T,CNT-EQUILIBRE_PIB,S15,P3,SO,RATIO,FE,...,France,France,Volumes aux prix de l'année précédente chaînés,Volumes chained at previous year prices,sans objet,not applicable,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no


In [ ]:
# Chargement ou extraction du PIB
fichier_pib = os.path.join(DATA_DIR, 'pib_france.csv')

if os.path.exists(fichier_pib):
    df_pib = pd.read_csv(fichier_pib)
    print(f"PIB chargé : {df_pib.shape[0]} lignes")
    print(f"Période : {df_pib['TIME_PERIOD'].min()} à {df_pib['TIME_PERIOD'].max()}")
else:
    idbanks_pib = series_pib['IDBANK'].tolist()
    df_pib = get_series(idbanks_pib)
    print(f"PIB extrait : {df_pib.shape}")

df_pib.head()

In [6]:
# Sauvegarde des données PIB
fichier_pib = os.path.join(DATA_DIR, 'pib_france.csv')
df_pib.to_csv(fichier_pib, index=False)
print(f"Fichier sauvegardé : {fichier_pib}")
print(f"Taille : {os.path.getsize(fichier_pib) / 1024:.1f} KB")

Fichier sauvegardé : /home/tinkerbell/Desktop/DataScientest/Compagnon Immobilier - Datascientest/Economie - Démographie/data/pib_france.csv
Taille : 20252.7 KB


### 3.2 IPC - Indice des Prix à la Consommation (Inflation)

In [7]:
# Dataset IPC
dataset_ipc = 'IPC-2015'

series_ipc = get_series_list(dataset_ipc)
print(f"Dataset : {dataset_ipc}")
print(f"Nombre de séries : {len(series_ipc)}")
series_ipc.head()

Dataset : IPC-2015
Nombre de séries : 4225


,DATASET,IDBANK,KEY,FREQ,INDICATEUR,FORME_VENTE,COICOP2016,PRIX_CONSO,NATURE,MENAGES_IPC,...,REF_AREA_label_fr,REF_AREA_label_en,UNIT_MEASURE_label_fr,UNIT_MEASURE_label_en,CORRECTION_label_fr,CORRECTION_label_en,BASIND_label_fr,BASIND_label_en,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en
0,IPC-2015,001765985,A.IPC.SO.045501.SO.POND.ENSEMBLE.FE.P10000.BRU...,A,IPC,SO,045501,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no
1,IPC-2015,001765984,A.IPC.SO.04550.SO.POND.ENSEMBLE.FE.P10000.BRUT...,A,IPC,SO,04550,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no
2,IPC-2015,001765983,A.IPC.SO.0455.SO.POND.ENSEMBLE.FE.P10000.BRUT....,A,IPC,SO,0455,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no
3,IPC-2015,001765982,A.IPC.SO.045491.SO.POND.ENSEMBLE.FE.P10000.BRU...,A,IPC,SO,045491,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no
4,IPC-2015,001765981,A.IPC.SO.04549.SO.POND.ENSEMBLE.FE.P10000.BRUT...,A,IPC,SO,04549,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no


In [ ]:
# Chargement ou extraction des données IPC
fichier_ipc = os.path.join(DATA_DIR, 'ipc_france.csv')

if os.path.exists(fichier_ipc):
    df_ipc = pd.read_csv(fichier_ipc)
    print(f"IPC chargé : {df_ipc.shape[0]} lignes")
    print(f"Période : {df_ipc['TIME_PERIOD'].min()} à {df_ipc['TIME_PERIOD'].max()}")
else:
    # Trop de séries (4225) - on filtre sur l'indice général
    series_filtrees = series_ipc[
        (series_ipc['FREQ'] == 'M') &
        (series_ipc['COICOP2016'].isin(['00', '0']))
    ]
    idbanks_ipc = series_filtrees['IDBANK'].tolist()
    df_ipc = get_series(idbanks_ipc)
    print(f"IPC extrait : {df_ipc.shape}")

df_ipc.head()

In [ ]:
# Sauvegarde IPC
fichier_ipc = os.path.join(DATA_DIR, 'ipc_france.csv')
df_ipc.to_csv(fichier_ipc, index=False)
print(f"Fichier sauvegardé : {fichier_ipc}")

### 3.3 Prix des Logements

In [ ]:
# Recherche des datasets sur les prix des logements
datasets_logement = rechercher_datasets('logement')
display(datasets_logement)

In [ ]:
# Indices des prix des logements (Notaires-INSEE)
dataset_prix_log = 'IPLA-IPLNA-2015'

series_prix = get_series_list(dataset_prix_log)
print(f"Séries disponibles : {len(series_prix)}")
series_prix.head(10)

In [ ]:
# Chargement ou extraction des prix des logements
fichier_prix = os.path.join(DATA_DIR, 'prix_logements_indices.csv')

if os.path.exists(fichier_prix):
    df_prix_log = pd.read_csv(fichier_prix)
    print(f"Prix logements chargé : {df_prix_log.shape[0]} lignes")
    print(f"Période : {df_prix_log['TIME_PERIOD'].min()} à {df_prix_log['TIME_PERIOD'].max()}")
else:
    idbanks_prix = series_prix['IDBANK'].tolist()
    df_prix_log = get_series(idbanks_prix)
    print(f"Prix logements extrait : {df_prix_log.shape}")

df_prix_log.head()

In [ ]:
# Sauvegarde
fichier_prix = os.path.join(DATA_DIR, 'prix_logements_indices.csv')
df_prix_log.to_csv(fichier_prix, index=False)
print(f"Fichier sauvegardé : {fichier_prix}")

---

## 4. Extraction des Données de Chômage

### 4.1 Chômage au niveau national

In [ ]:
# Dataset chômage
dataset_chomage = 'CHOMAGE-TRIM-NATIONAL'

series_chomage = get_series_list(dataset_chomage)
print(f"Dataset : {dataset_chomage}")
print(f"Nombre de séries : {len(series_chomage)}")
series_chomage.head()

In [ ]:
# Chargement ou extraction du chômage national
fichier_chomage = os.path.join(DATA_DIR, 'chomage_series.csv')

if os.path.exists(fichier_chomage):
    df_chomage = pd.read_csv(fichier_chomage)
    print(f"Chômage national chargé : {df_chomage.shape[0]} lignes")
    print(f"Période : {df_chomage['TIME_PERIOD'].min()} à {df_chomage['TIME_PERIOD'].max()}")
else:
    idbanks_chomage = series_chomage['IDBANK'].tolist()
    df_chomage = get_series(idbanks_chomage)
    print(f"Chômage national extrait : {df_chomage.shape}")

df_chomage.head()

In [ ]:
# Sauvegarde
fichier_chomage = os.path.join(DATA_DIR, 'chomage_series.csv')
df_chomage.to_csv(fichier_chomage, index=False)
print(f"Fichier sauvegardé : {fichier_chomage}")

### 4.2 Chômage par département (Taux de chômage localisé)

In [ ]:
# Taux de chômage localisé (par département/région)
dataset_chomage_loc = 'TAUX-CHOMAGE'

series_chomage_loc = get_series_list(dataset_chomage_loc)
print(f"Dataset : {dataset_chomage_loc}")
print(f"Nombre de séries : {len(series_chomage_loc)}")
series_chomage_loc.head(10)

In [ ]:
# Chargement ou extraction du chômage localisé
fichier_chomage_dept = os.path.join(DATA_DIR, 'chomage_departements_complet.csv')

if os.path.exists(fichier_chomage_dept):
    df_chomage_dept = pd.read_csv(fichier_chomage_dept)
    print(f"Chômage localisé chargé : {df_chomage_dept.shape[0]} lignes")
    print(f"Période : {df_chomage_dept['TIME_PERIOD'].min()} à {df_chomage_dept['TIME_PERIOD'].max()}")
else:
    idbanks_chomage_loc = series_chomage_loc['IDBANK'].tolist()
    df_chomage_dept = get_series(idbanks_chomage_loc)
    print(f"Chômage localisé extrait : {df_chomage_dept.shape}")

df_chomage_dept.head()

In [ ]:
# Sauvegarde chômage départements
fichier_chomage_dept = os.path.join(DATA_DIR, 'chomage_departements_complet.csv')
df_chomage_dept.to_csv(fichier_chomage_dept, index=False)
print(f"Fichier sauvegardé : {fichier_chomage_dept}")
print(f"Taille : {os.path.getsize(fichier_chomage_dept) / 1024:.1f} KB")

---

## 5. Extraction des Données Locales (Départements)

### 5.1 Population par département

In [ ]:
# Liste des codes départements français
codes_departements = [f"{i:02d}" for i in range(1, 96) if i != 20]  # 01 à 95 sauf 20
codes_departements.extend(['2A', '2B'])  # Corse
codes_departements.extend(['971', '972', '973', '974', '976'])  # DOM

print(f"Nombre de départements : {len(codes_departements)}")
print(f"Premiers codes : {codes_departements[:10]}")

In [ ]:
# Chargement des données de population
fichier_pop = os.path.join(DATA_DIR, 'population_departements.csv')

if os.path.exists(fichier_pop):
    df_pop = pd.read_csv(fichier_pop)
    print(f"Population chargé : {df_pop.shape[0]} lignes")
    display(df_pop.head())
else:
    print("Extraction population via get_local_data...")
    try:
        df_pop = get_local_data(
            variables='SEXE',
            dataset_version='GEO2024RP2021',
            nivgeo='DEP',
            geocodes=codes_departements[:5]
        )
        print(f"Population extrait : {df_pop.shape}")
    except Exception as e:
        print(f"Erreur : {e}")

### 5.2 Revenus par département

In [ ]:
# Les données de revenus sont souvent dans des datasets spécifiques
# FILOSOFI (Fichier Localisé Social et Fiscal)

datasets_revenus = rechercher_datasets('revenu')
display(datasets_revenus)

In [ ]:
# Chargement des données de revenus
fichier_revenus = os.path.join(DATA_DIR, 'revenus_departements.csv')

if os.path.exists(fichier_revenus):
    df_revenus = pd.read_csv(fichier_revenus)
    print(f"Revenus chargé : {df_revenus.shape[0]} lignes")
    display(df_revenus.head())
else:
    print("Fichier revenus_departements.csv non trouvé")
    print("Extraction via get_local_data (peut être long)...")
    try:
        df_revenus = get_local_data(
            variables='REVENU',
            dataset_version='GEO2024FILOSOFI2021',
            nivgeo='DEP',
            geocodes=codes_departements
        )
        print(f"Revenus extrait : {df_revenus.shape}")
    except Exception as e:
        print(f"Erreur : {e}")

---

## 6. Récapitulatif des Fichiers Extraits

In [ ]:
# Récapitulatif de tous les fichiers dans le dossier data
print("=" * 80)
print("FICHIERS DE DONNÉES DISPONIBLES")
print("=" * 80)

fichiers_csv = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
fichiers_csv.sort()

total_size = 0
for f in fichiers_csv:
    chemin = os.path.join(DATA_DIR, f)
    size = os.path.getsize(chemin) / 1024
    total_size += size
    
    # Charger pour avoir le nombre de lignes
    df = pd.read_csv(chemin)
    print(f"{f:45} | {size:8.1f} KB | {len(df):7} lignes | {len(df.columns):2} cols")

print("=" * 80)
print(f"Total : {len(fichiers_csv)} fichiers | {total_size/1024:.1f} MB")

---

## 7. Fonction Utilitaire d'Extraction

Fonction réutilisable pour extraire et sauvegarder des données :

In [ ]:
def extraire_et_sauvegarder(dataset_id, nom_fichier, dossier=DATA_DIR):
    """
    Extrait un dataset INSEE et le sauvegarde en CSV.
    
    Paramètres:
        dataset_id (str): Identifiant du dataset INSEE
        nom_fichier (str): Nom du fichier de sortie (avec .csv)
        dossier (str): Dossier de destination
    
    Retourne:
        DataFrame ou None en cas d'erreur
    """
    try:
        print(f"Extraction de {dataset_id}...")
        
        # 1. Récupérer la liste des séries et leurs IDBANK
        series_list = get_series_list(dataset_id)
        idbanks = series_list['IDBANK'].tolist()
        print(f"  -> {len(idbanks)} séries trouvées")
        
        # 2. Extraire les données
        df = get_series(idbanks)
        
        # 3. Sauvegarde
        chemin = os.path.join(dossier, nom_fichier)
        df.to_csv(chemin, index=False)
        
        print(f"  -> {len(df)} lignes extraites")
        print(f"  -> Sauvegardé : {nom_fichier}")
        
        return df
        
    except Exception as e:
        print(f"  -> Erreur : {e}")
        return None

In [ ]:
# Exemple d'utilisation
# extraire_et_sauvegarder('CNT-2020-PIB-EQB-RF', 'pib_france_test.csv')

---

## Conclusion

Données extraites via l'API INSEE :

| Catégorie | Dataset | Fichier |
|-----------|---------|---------|
| PIB | CNT-2020-PIB-EQB-RF | pib_france.csv |
| Inflation | IPC-2015 | ipc_france.csv |
| Prix logements | IPLA-IPLNA-2015 | prix_logements_indices.csv |
| Chômage national | CHOMAGE-TRIM-NATIONAL | chomage_series.csv |
| Chômage localisé | TAUX-CHOMAGE | chomage_departements_complet.csv |

**Notebook suivant** : `03_nettoyage_donnees.ipynb`